<a href="https://colab.research.google.com/github/rameshavinash94/POEM-NLP/blob/main/NLP_POEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALL SPACY ENGLISH LANGUAGE MODEL AND TRANSFORMERS LIBRARY**

In [ ]:
#install spacy medium english language model to use
! python -m spacy download en_core_web_md
! pip install transformers

In [ ]:
exit()

**IMPORTING LIBRARIES**

In [ ]:
import spacy
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from numba import jit
import numpy as np

**SUPRESS WARNING**

In [ ]:
#just to supress warnings
import warnings
warnings.filterwarnings("ignore")

**ADD GPU**

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**POEM CLASS WITH ALL FUNCTIONALITY**

In [ ]:
import tensorflow as tf

class Poem:
  def __init__(self):
       self.nlp = spacy.load('en_core_web_md')
       self.doc1_pos={'PROPN':set(), 'NUM':set(),'VERB':set(),'NOUN':set(),'ADJ':set()}
       self.doc2_pos={'PROPN':set(), 'NUM':set(),'VERB':set(),'NOUN':set(),'ADJ':set()}
       self.doc1_mappings={}
       self.doc2_mappings={}
       self.poet1_details={}
       self.poet2_details={}
    
  def UserInputbyPoet(self,poemsperpoet=10,no_of_poets=2):
        '''
        This Function is to get user query for the poet name and list of poems to scrap.
        '''
        self.poet1 = input('Enter the poet name to scrap {n} poems of him:'.format(n=poemsperpoet))
        self.poet2 = input('Enter the poet name to scrap {n} poems of him:'.format(n=poemsperpoet))
        self.poemsperpoet = poemsperpoet
        
        if not (self.poet1 or self.poet2):
          raise Exception('User input is empty or wrong please retype properly')
        
        #user input preprocessing per api
        self.poet1= self.poet1.title()
        self.poet2= self.poet2.title()
        
        return self.poet1,self.poet2,self.poemsperpoet
    
  def Scrapperbypoet(self,poemsperpoet=10):
    poet1={'poet':[],'title':[],'poem':[]}
    poet2={'poet':[],'title':[],'poem':[]}
    cookies = {
        'z': '1',
        'jt': '1',
        'tok': 'WzYwNTMxNzksImQ3ODVlZTdjMjk4YjQzODY5OWIyYmI1NTRiZjYxYWQzIl0%3D--62c210b76358a3e254c5f0219b490fd852676901',
        '_N': 'OWIwK3d4VUV0ZWQrZk1sZWxBWHV3TGIzQ210NDhkaWRWTXFPUlB4bldMY1BIVkMrT2dRZ2dnajBseEJ3NzRxZjRGZ2c0a2tXSVhEdThPZzZKOE9UK2pJanFRV094N2R2K1lUajB6MzE0MUcwWWJhbUNLTUpDZ2toVEd2dFo3OUtMZHFxYUpmb0ZsOG1aUzFpTjYyQy8zU0h4a04yQTBGUVZiY1gyRGwxR253ZUtIalg4Sk5xZnZNYTE2a2pEQjgxQk9aKytldHhreThUMmlLbE01MWRyUGttUWp3MXBxc0doZkhuVUdwcURBcVNRRmRSSmljVkJOMi8vSjRnUWJqRGNwaEhzYURJZjRhM1hOOEhGdG4yS0FNOTA5aXA3MHVjMVpMZmJCZ1RaVTFhZTNRQmhjWHZmdTJucUprczlZZmlURHcwVjdIREZLZm5JVm4rekljNm4vQVRlem50Yy9SM2R5ckNCYzZwUzViT1F2R0UvSjNiRnlaQ1F1QS9iZEtKRzNDcFpHWE0za3FZdzgyQjd0WFBoZ2FuTUZ6V1hidE8rUFpvd0htWTR2bzY3elNLZ3NqVUhVcWJxV25uRUt5NHFCS2ptSWlpQm9uUnRtUWZoK0dYMVJDQnc4cERKaCtkY29lM0JadGhXYU9rQ1Fvdkt4UVdaM1RqWmMyVzFqYlJhRnloZ0pHYlVsL2cwZm5uOEZXV2JLQ1JWODNrM3FZc3FDL3haWWl2cVpBPS0tQzQzVGljYVJtY3hIdUNpeFYrRmk3UT09--1af7abe737f99eccc42280051a1aaf758b3cc69a',
    }
    headers = {
        'authority': 'allpoetry.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-language': 'en-US,en;q=0.9',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'z=1; jt=1; tok=WzYwNTMxNzksImQ3ODVlZTdjMjk4YjQzODY5OWIyYmI1NTRiZjYxYWQzIl0%3D--62c210b76358a3e254c5f0219b490fd852676901; _N=OWIwK3d4VUV0ZWQrZk1sZWxBWHV3TGIzQ210NDhkaWRWTXFPUlB4bldMY1BIVkMrT2dRZ2dnajBseEJ3NzRxZjRGZ2c0a2tXSVhEdThPZzZKOE9UK2pJanFRV094N2R2K1lUajB6MzE0MUcwWWJhbUNLTUpDZ2toVEd2dFo3OUtMZHFxYUpmb0ZsOG1aUzFpTjYyQy8zU0h4a04yQTBGUVZiY1gyRGwxR253ZUtIalg4Sk5xZnZNYTE2a2pEQjgxQk9aKytldHhreThUMmlLbE01MWRyUGttUWp3MXBxc0doZkhuVUdwcURBcVNRRmRSSmljVkJOMi8vSjRnUWJqRGNwaEhzYURJZjRhM1hOOEhGdG4yS0FNOTA5aXA3MHVjMVpMZmJCZ1RaVTFhZTNRQmhjWHZmdTJucUprczlZZmlURHcwVjdIREZLZm5JVm4rekljNm4vQVRlem50Yy9SM2R5ckNCYzZwUzViT1F2R0UvSjNiRnlaQ1F1QS9iZEtKRzNDcFpHWE0za3FZdzgyQjd0WFBoZ2FuTUZ6V1hidE8rUFpvd0htWTR2bzY3elNLZ3NqVUhVcWJxV25uRUt5NHFCS2ptSWlpQm9uUnRtUWZoK0dYMVJDQnc4cERKaCtkY29lM0JadGhXYU9rQ1Fvdkt4UVdaM1RqWmMyVzFqYlJhRnloZ0pHYlVsL2cwZm5uOEZXV2JLQ1JWODNrM3FZc3FDL3haWWl2cVpBPS0tQzQzVGljYVJtY3hIdUNpeFYrRmk3UT09--1af7abe737f99eccc42280051a1aaf758b3cc69a',
        'if-none-match': 'W/"6ebecc9a316dba275701ba119fa8ae21"',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'sec-gpc': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36',
    }

    poet1_poems = requests.get('https://allpoetry.com/classics/alpha/{poet1}'.format(poet1=self.poet1), headers=headers, cookies=cookies)
    poet2_poems = requests.get('https://allpoetry.com/classics/alpha/{poet2}'.format(poet2=self.poet2), headers=headers, cookies=cookies)

    
    doc1=BeautifulSoup(poet1_poems.text,'html.parser')
    doc2=BeautifulSoup(poet2_poems.text,'html.parser')
    
    poet1_details={'poet':[],'title':[],'poem':[],'poem_url':[]}
    poet2_details={'poet':[],'title':[],'poem':[],'poem_url':[]}

    for ul in doc1.find_all('ul',attrs={"class" : "gapped my2"}):
      ahref = ul.find_all("a",href=True)[:poemsperpoet]
      for x in ahref:
        tmp=self.Scrapperbypoem(x['href'])
        poet1_details['poet'].append(tmp['poet'])
        poet1_details['title'].append(x.text)
        poet1_details['poem'].append(tmp['poem'])
        poet1_details['poem_url'].append('https://allpoetry.com{url}'.format(url=x['href']))

    for ul in doc2.find_all('ul',attrs={"class" : "gapped my2"}):
      ahref = ul.find_all("a",href=True)[:poemsperpoet]
      for x in ahref:
        tmp=self.Scrapperbypoem(x['href'])
        poet2_details['poet'].append(tmp['poet'])
        poet2_details['title'].append(x.text)
        poet2_details['poem'].append(tmp['poem'])
        poet2_details['poem_url'].append('https://allpoetry.com{url}'.format(url=x['href']))

    self.poet1_details=poet1_details
    self.poet2_details=poet2_details
    return self.poet1_details,self.poet2_details

  def Scrapperbypoem(self,poem):
      '''
        #Iterate and Scrap data to get poem and author name and store in Df
      '''
      poem_info={'poet':'','poem':''}
      url='https://allpoetry.com{iter}'.format(iter=poem)
      html_content = requests.get(url).text
      doc=BeautifulSoup(html_content,'html.parser')

      for div in doc.find_all('div',attrs={"class" : re.compile("^bio*")})[0]:
          ahref = div.find_all("a",attrs={"class": "u nocolor"})
          for x in ahref:
            if x!=None:
                  poem_info['poet']= x.text

      for div in doc.find_all('div',attrs={"class" : re.compile("^orig")}):
          poem_info['poem']= div.text
      
      return poem_info

  def Convert_df(self):
    '''
    This function converts the peom info to DF.
    '''
    self.poet_1_details_df = pd.DataFrame(self.poet1_details)
    self.poet_2_details_df = pd.DataFrame(self.poet2_details)

  def lemmatization(self,corpus):
    '''
    This Function removes stop words and lemmatizes the provided corpus
    :return: Stop word removed & Canonical form corpus (canonical)
    '''
    doc=self.nlp(corpus)
    temp=[]
    for token in doc:
        if not token.is_stop:
            temp.append(token.lemma_)

    canonical = " ".join(temp)

    return canonical
  
  def remove_sp(self,corpus):
    '''
    this function is used to remove special characters from the corpus
    '''
    import re
    return re.sub(r"[^a-zA-Z0-9]+", ' ', corpus)
  
  def PreprocessPoems(self,lematization=True):
    '''
    This Function is to preprocess user input,calls lemmatization function and extract only the nceesary POS if requested.
    '''
    if lematization==True:
      self.poet_1_details_df['lemma_stop_word_removal'] = self.poet_1_details_df['poem'].map(lambda x : self.lemmatization(x))
      self.poet_2_details_df['lemma_stop_word_removal'] = self.poet_2_details_df['poem'].map(lambda x : self.lemmatization(x))
      self.poet_1_details_df['cleaned'] = self.poet_1_details_df['lemma_stop_word_removal'].map(lambda x: self.remove_sp(x))
      self.poet_2_details_df['cleaned'] = self.poet_2_details_df['lemma_stop_word_removal'].map(lambda x: self.remove_sp(x))
    else:
      self.poet_1_details_df['cleaned'] = self.poet_1_details_df['poem'].map(lambda x: self.remove_sp(x))
      self.poet_2_details_df['cleaned'] = self.poet_2_details_df['poem'].map(lambda x: self.remove_sp(x))

  def find_Pos(self):
    '''
    this function is used to find pos
    '''
    for x in self.poet_1_details_df['cleaned']:
      doc1=self.nlp(x)
      for token in doc1:
        if token.pos_ == 'PROPN' and len(token)>2:
          self.doc1_pos['PROPN'].add(token.text)
        
        elif token.pos_ == 'NUM':
          self.doc1_pos['NUM'].add(token.text)
        
        elif token.pos_ == 'VERB':
          self.doc1_pos['VERB'].add(token.text)
        
        elif token.pos_ == 'NOUN':
          self.doc1_pos['NOUN'].add(token.text)
        
        elif token.pos_ == 'ADJ':
          self.doc1_pos['ADJ'].add(token.text)
  
    for x in self.poet_2_details_df['cleaned']:
      doc2=self.nlp(x)
      for token in doc2:
        if token.pos_ == 'PROPN' and len(token)>2:
          self.doc2_pos['PROPN'].add(token.text)
        
        elif token.pos_ == 'NUM':
          self.doc2_pos['NUM'].add(token.text)
        
        elif token.pos_ == 'VERB':
          self.doc2_pos['VERB'].add(token.text)
        
        elif token.pos_ == 'NOUN':
          self.doc2_pos['NOUN'].add(token.text)
        
        elif token.pos_ == 'ADJ':
          self.doc2_pos['ADJ'].add(token.text)
    
    return self.doc1_pos,self.doc2_pos

  def loopover(self,set1,set2):
    with tf.device('/device:GPU:0'):
      '''
      just loop over 2 sets and find cosine similarity with gpu
      '''
      # @jit(nopython=True)
      # def cosine_similarity_numba(u:np.ndarray, v:np.ndarray):
      #   assert(u.shape[0] == v.shape[0])
      #   uv = 0
      #   uu = 0
      #   vv = 0
      #   for i in range(u.shape[0]):
      #       uv += u[i]*v[i]
      #       uu += u[i]*u[i]
      #       vv += v[i]*v[i]
      #   cos_theta = 1
      #   if uu!=0 and vv!=0:
      #       cos_theta = uv/np.sqrt(uu*vv)
      #   return cos_theta

      tmp1=set1
      tmp2=set2
      doc_mappings={}
      for x in tmp1:
        w1=self.nlp(x)
        max=-1
        for y in tmp2:
          w2=self.nlp(y)
          simliariy_score=w1.similarity(w2)
          if simliariy_score > 0.6:
            doc_mappings[x]=[y,simliariy_score]
            break
          elif simliariy_score > max:
            max=simliariy_score
            doc_mappings[x]=[y,max]
        
        if max==0.0:
          doc_mappings[x]=[x,None]
        
      print(doc_mappings)
      return doc_mappings

  def POS_ComputeSimilarity(self):
    '''
    This function computes cosine similarity
    '''
    with tf.device('/device:GPU:0'):
      for x in self.doc1_pos:
          self.doc1_mappings.update(self.loopover(self.doc1_pos[x],self.doc2_pos[x]))
      
      for x in self.doc2_pos:
          self.doc2_mappings.update(self.loopover(self.doc2_pos[x],self.doc1_pos[x]))

  def Create_new_Poems(self):
    '''
    This functino creates new poems based on the provided one's.
    '''
    self.new_poem1_poem2=[]
    self.new_poem2_poem1=[]
    for x in self.poet_1_details_df['lemma_stop_word_removal']:
      new_poem3=str(x)
      for key, value in self.doc1_mappings.items():
      # Replace key POS with value POS in string
        new_poem3 = re.sub(r'\b{key}\b'.format(key=str(key)), str(value[0]),new_poem3)
      self.new_poem1_poem2.append(str(new_poem3))
      
    for x in self.poet_2_details_df['lemma_stop_word_removal']:
      new_poem4=str(x)
      for key, value in self.doc2_mappings.items():
      # Replace key POS with value POS in string
        new_poem4 = re.sub(r'\b{key}\b'.format(key=str(key)), str(value[0]),new_poem4)
      
      self.new_poem2_poem1.append(str(new_poem4))
    
    self.poet_1_details_df['new_poem'] = self.new_poem1_poem2
    self.poet_2_details_df['new_poem'] = self.new_poem2_poem1

**CREATE OBJECT FOR THE CLASS**

In [ ]:
#create object for the class
tmp = Poem()

**GET INPUT FOR 2 POETS**

In [ ]:
# Poet1 - Edward Dyer   
# Poet2 - Walt Whitman

In [ ]:
#GET USER INPUT
tmp.UserInputbyPoet()

Enter the poet name to scrap 10 poems of him:billy collins
Enter the poet name to scrap 10 poems of him:Mary J Oliver


('Billy Collins', 'Mary J Oliver', 10)

**RUN SCRAPPER TO EXTRACT POEM FROM ALLPOETRY**

In [ ]:
#Call scrapper function to scrap poems from both poets
tmp.Scrapperbypoet()

({'poem': ["\r\n\tThe neighbors' dog will not stop barking.\r\n\tHe is barking the same high, rhythmic bark\r\n\tthat he barks every time they leave the house.\r\n\tThey must switch him on on their way out.\n\r\n\t\xa0\n\r\n\tThe neighbors' dog will not stop barking.\r\n\tI close all the windows in the house\r\n\tand put on a Beethoven symphony full blast\r\n\tbut I can still hear him muffled under the music,\r\n\tbarking, barking, barking,\n\r\n\t\xa0\n\r\n\tand now I can see him sitting in the orchestra,\r\n\this head raised confidently as if Beethoven\r\n\thad included a part for barking dog.\n\r\n\t\xa0\n\r\n\tWhen the record finally ends he is still barking,\r\n\tsitting there in the oboe section barking,\r\n\this eyes fixed on the conductor who is\r\n\tentreating him with his baton\n\r\n\t\xa0\n\r\n\twhile the other musicians listen in respectful\r\n\tsilence to the famous barking dog solo,\r\n\tthat endless coda that first established\r\n\tBeethoven as an innovative genius.",
  

**CONVERT THE INFO TO DF TO EASIER VISUALIZATION**

In [ ]:
#convert to df both poem1, poem2
tmp.Convert_df()

In [ ]:
#view df1
tmp.poet_1_details_df

,poet,title,poem,poem_url
0,Billy Collins,Another Reason Why I Don't Keep A Gun In The H...,\r\n\tThe neighbors' dog will not stop barking...,https://allpoetry.com/Another-Reason-Why-I-Don...
1,Billy Collins,By A Swimming Pool Outside Syracusa,All afternoon I have been struggling to commun...,https://allpoetry.com/poem/14327199-By-A-Swimm...
2,Billy Collins,Candle Hat,\r\n\tIn most self-portraits it is the face th...,https://allpoetry.com/Candle-Hat
3,Billy Collins,Child Development,As sure as prehistoric fish grew legs\nand sau...,https://allpoetry.com/poem/14327208-Child-Deve...
4,Billy Collins,Christmas Sparrow,The first thing I heard this morning\nwas a so...,https://allpoetry.com/poem/14327179-Christmas-...
5,Billy Collins,Consolation,\r\n\tHow agreeable it is not to be touring It...,https://allpoetry.com/poem/9742271-Consolation...
6,Billy Collins,Dear Reader,"Baudelaire considers you his brother, and Fiel...",https://allpoetry.com/poem/14327192-Dear-Reade...
7,Billy Collins,Embrace,You know the parlor trick.\nwrap your arms aro...,https://allpoetry.com/poem/14327198-Embrace-by...
8,Billy Collins,Fishing on the Susquehanna in July,\r\n\tI have never been fishing on the Susqueh...,https://allpoetry.com/Fishing-on-the-Susquehan...
9,Billy Collins,Flames,Smokey the Bear heads\ninto the autumn woods\n...,https://allpoetry.com/poem/14327207-Flames-by-...


In [ ]:
#view df2
tmp.poet_2_details_df

,poet,title,poem,poem_url
0,Mary J Oliver,Breakage,I go down to the edge of the sea.\nHow everyth...,https://allpoetry.com/poem/15374794-Breakage-b...
1,Mary J Oliver,Flare,"1.\n\nWelcome to the silly, comforting poem.\n...",https://allpoetry.com/poem/15374212-Flare-by-M...
2,Mary J Oliver,Happiness,In the afternoon I watched\nthe she-bear; she ...,https://allpoetry.com/poem/15374236-Happiness-...
3,Mary J Oliver,In Blackwater woods,"Look, the trees\nare turning\ntheir own bodies...",https://allpoetry.com/poem/15374219-In-Blackwa...
4,Mary J Oliver,Next time,Next time what I'd do is look at\nthe earth be...,https://allpoetry.com/poem/15374239-Next-time-...
5,Mary J Oliver,Song for Autumn,Don’t you imagine the leaves dream now\nhow co...,https://allpoetry.com/poem/15374796-Song-for-A...
6,Mary J Oliver,The Journey,"One day you finally knew\nwhat you had to do, ...",https://allpoetry.com/poem/15374221-The-Journe...
7,Mary J Oliver,The summer day,"Who made the world?\nWho made the swan, and th...",https://allpoetry.com/poem/15374240-The-summer...
8,Mary J Oliver,"Where Does the Dance Begin, Where Does It End?","Don't call this world adorable, or useful, tha...",https://allpoetry.com/poem/15374244-Where-Does...
9,Mary J Oliver,Wild geese,You do not have to be good.\nYou do not have t...,https://allpoetry.com/poem/15374223-Wild-geese...


**PERFORM NLP PREPROCESSING - STOP WORD REMOVAL/LEMMATIZATION/CLEAN THE CORPUS**

In [ ]:
#run preporcessing function, for now have set lemma to false
tmp.PreprocessPoems()

In [ ]:
tmp.poet_1_details_df

,poet,title,poem,poem_url,lemma_stop_word_removal,cleaned
0,Billy Collins,Another Reason Why I Don't Keep A Gun In The H...,\r\n\tThe neighbors' dog will not stop barking...,https://allpoetry.com/Another-Reason-Why-I-Don...,\r\n\t neighbor ' dog stop bark . \r\n\t bark ...,neighbor dog stop bark bark high rhythmic bar...
1,Billy Collins,By A Swimming Pool Outside Syracusa,All afternoon I have been struggling to commun...,https://allpoetry.com/poem/14327199-By-A-Swimm...,afternoon struggle communicate Italian Roberto...,afternoon struggle communicate Italian Roberto...
2,Billy Collins,Candle Hat,\r\n\tIn most self-portraits it is the face th...,https://allpoetry.com/Candle-Hat,\r\n\t self - portrait face dominate : \r\n\t ...,self portrait face dominate Cezanne pair eye ...
3,Billy Collins,Child Development,As sure as prehistoric fish grew legs\nand sau...,https://allpoetry.com/poem/14327208-Child-Deve...,sure prehistoric fish grow leg \n saunter beac...,sure prehistoric fish grow leg saunter beach f...
4,Billy Collins,Christmas Sparrow,The first thing I heard this morning\nwas a so...,https://allpoetry.com/poem/14327179-Christmas-...,"thing hear morning \n soft , insistent rustle ...",thing hear morning soft insistent rustle rapid...
5,Billy Collins,Consolation,\r\n\tHow agreeable it is not to be touring It...,https://allpoetry.com/poem/9742271-Consolation...,"\r\n\t agreeable tour Italy summer , \r\n\t wa...",agreeable tour Italy summer wander city ascen...
6,Billy Collins,Dear Reader,"Baudelaire considers you his brother, and Fiel...",https://allpoetry.com/poem/14327192-Dear-Reade...,"Baudelaire consider brother , Fielding call pa...",Baudelaire consider brother Fielding call para...
7,Billy Collins,Embrace,You know the parlor trick.\nwrap your arms aro...,https://allpoetry.com/poem/14327198-Embrace-by...,know parlor trick . \n wrap arm body \n look l...,know parlor trick wrap arm body look like embr...
8,Billy Collins,Fishing on the Susquehanna in July,\r\n\tI have never been fishing on the Susqueh...,https://allpoetry.com/Fishing-on-the-Susquehan...,\r\n\t fish Susquehanna \r\n\t river matter \r...,fish Susquehanna river matter perfectly hones...
9,Billy Collins,Flames,Smokey the Bear heads\ninto the autumn woods\n...,https://allpoetry.com/poem/14327207-Flames-by-...,smokey Bear head \n autumn wood \n red gasolin...,smokey Bear head autumn wood red gasoline box ...


In [ ]:
tmp.poet_2_details_df

,poet,title,poem,poem_url,lemma_stop_word_removal,cleaned
0,Mary J Oliver,Breakage,I go down to the edge of the sea.\nHow everyth...,https://allpoetry.com/poem/15374794-Breakage-b...,edge sea . \n shine morning light ! \n cusp wh...,edge sea shine morning light cusp whelk break ...
1,Mary J Oliver,Flare,"1.\n\nWelcome to the silly, comforting poem.\n...",https://allpoetry.com/poem/15374212-Flare-by-M...,"1 . \n\n welcome silly , comfort poem . \n\n s...",1 welcome silly comfort poem sunrise red rinse...
2,Mary J Oliver,Happiness,In the afternoon I watched\nthe she-bear; she ...,https://allpoetry.com/poem/15374236-Happiness-...,afternoon watch \n - bear ; look \n secret bin...,afternoon watch bear look secret bin sweetness...
3,Mary J Oliver,In Blackwater woods,"Look, the trees\nare turning\ntheir own bodies...",https://allpoetry.com/poem/15374219-In-Blackwa...,"look , tree \n turn \n body \n pillar \n\n lig...",look tree turn body pillar light give rich fra...
4,Mary J Oliver,Next time,Next time what I'd do is look at\nthe earth be...,https://allpoetry.com/poem/15374239-Next-time-...,time look \n earth say . stop \n go house \n e...,time look earth say stop go house emperor minu...
5,Mary J Oliver,Song for Autumn,Don’t you imagine the leaves dream now\nhow co...,https://allpoetry.com/poem/15374796-Song-for-A...,imagine leave dream \n comfortable touch \n ea...,imagine leave dream comfortable touch earth in...
6,Mary J Oliver,The Journey,"One day you finally knew\nwhat you had to do, ...",https://allpoetry.com/poem/15374221-The-Journe...,"day finally know \n , begin , \n voice \n keep...",day finally know begin voice keep shout bad ad...
7,Mary J Oliver,The summer day,"Who made the world?\nWho made the swan, and th...",https://allpoetry.com/poem/15374240-The-summer...,"world ? \n swan , black bear ? \n grasshopper ...",world swan black bear grasshopper grasshopper ...
8,Mary J Oliver,"Where Does the Dance Begin, Where Does It End?","Don't call this world adorable, or useful, tha...",https://allpoetry.com/poem/15374244-Where-Does...,"world adorable , useful , . \n frisky , theate...",world adorable useful frisky theater fair wind...
9,Mary J Oliver,Wild geese,You do not have to be good.\nYou do not have t...,https://allpoetry.com/poem/15374223-Wild-geese...,"good . \n walk knee \n mile desert , repent . ...",good walk knee mile desert repent let soft ani...


**FIND POS**

In [ ]:
doc1pos,doc2pos = tmp.find_Pos()

In [ ]:
doc1pos

{'ADJ': {'Dumb',
  'Italian',
  'abbeys',
  'agreeable',
  'american',
  'appalling',
  'attentive',
  'bad',
  'big',
  'blue',
  'brim',
  'brown',
  'careless',
  'close',
  'cold',
  'dark',
  'decorate',
  'dense',
  'dopey',
  'dormant',
  'eager',
  'easy',
  'elbow',
  'endless',
  'errant',
  'extraordinary',
  'familiar',
  'famous',
  'fatuous',
  'fit',
  'flat',
  'flush',
  'giant',
  'gleam',
  'great',
  'green',
  'high',
  'honest',
  'hothead',
  'hungry',
  'ignorant',
  'innovative',
  'insistent',
  'invective',
  'irregular',
  'light',
  'little',
  'local',
  'loud',
  'lucky',
  'male',
  'mature',
  'metallic',
  'new',
  'old',
  'open',
  'precinct',
  'prehistoric',
  'professional',
  'quiet',
  'rapid',
  'rare',
  'ready',
  'respectful',
  'rhythmic',
  'right',
  'sarcophagus',
  'screwy',
  'sick',
  'silent',
  'simple',
  'small',
  'soft',
  'spacious',
  'spiky',
  'straight',
  'sudden',
  'tall',
  'thin',
  'tight',
  'upstream',
  'vernacular

In [ ]:
doc2pos

{'ADJ': {'able',
  'adorable',
  'bad',
  'bearded',
  'beautiful',
  'better',
  'black',
  'blue',
  'center',
  'clean',
  'comfortable',
  'complicated',
  'cool',
  'curved',
  'dark',
  'deep',
  'deeper',
  'delicate',
  'diligent',
  'drunk',
  'early',
  'endless',
  'enormous',
  'exciting',
  'fair',
  'frisky',
  'good',
  'grave',
  'gray',
  'green',
  'hard',
  'harsh',
  'heavy',
  'high',
  'hollow',
  'last',
  'late',
  'light',
  'little',
  'live',
  'lonely',
  'long',
  'longing',
  'lovely',
  'messy',
  'modest',
  'muscular',
  'nameless',
  'natured',
  'net',
  'nothingness',
  'old',
  'open',
  'orderliness',
  'outward',
  'pale',
  'pink',
  'plain',
  'poor',
  'precious',
  'restful',
  'rich',
  'secret',
  'sheer',
  'silly',
  'sleepy',
  'small',
  'soft',
  'sound',
  'sticky',
  'stiff',
  'straight',
  'strange',
  'sweet',
  'tall',
  'tambourine',
  'tattered',
  'terrible',
  'thick',
  'thin',
  'unforgettable',
  'unpleasant',
  'untidy',
 

**COMPUTE SIMILARITY OF POS**

In [ ]:
tmp.POS_ComputeSimilarity()

{'Susquehanna': ['Europe', 0.44458150646467176], 'record': ['time', 0.4278775824073826], 'Sillypants': ['Sillypants', None], 'angel': ['God', 0.5136990257769904], 'concert': ['day', 0.35907981603034017], 'Samuel': ['mockingbird', 0.43347387032833556], 'Philadelphia': ['Europe', 0.404120377208502], 'memorize': ['poem', 0.3100467386244252], 'holly': ['jingle', 0.42124452459636075], 'Goopyhead': ['Goopyhead', None], 'succession': ['cusp', 0.35918764150787547], 'Beginners': ['great', 0.3132222177025964], 'eye': ['head', 0.5111818214852921], 'Enlightenment': ['everlasting', 0.5022305458482222], 'puzzle': ['dream', 0.3102005507084049], 'Bear': ['fox', 0.4937987093039136], 'Cezanne': ['mockingbird', 0.29851423169845465], 'bush': ['fox', 0.46796537597547727], 'Rembrant': ['tuffet', 0.39933969681712894], 'miss': ['hurry', 0.5012191512215836], 'Elba': ['mockingbird', 0.34876125002394576], 'Timmy': ['fox', 0.3120472833134427], 'box': ['red', 0.40586904579011707], 'bill': ['education', 0.340078054

In [ ]:
tmp.doc1_mappings

{'Baudelaire': ['mockingbird', 0.3210952234634215],
 'Bear': ['fox', 0.4937987093039136],
 'Beethoven': ['mockingbird', 0.333755009119292],
 'Beginners': ['great', 0.3132222177025964],
 'Big': ['sizzling', 0.5680608668918296],
 'Blinding': ['stare', 0.3897518063271421],
 'Bologna': ['scallop', 0.5245861323387164],
 'Cezanne': ['mockingbird', 0.29851423169845465],
 'Dopeheads': ['Dopeheads', None],
 'Dot': ['blue', 0.4339867192407223],
 'Dumb': ['silly', 0.7799713890983614],
 'Elba': ['mockingbird', 0.34876125002394576],
 'English': ['Europe', 0.45332710389385816],
 'Enlightenment': ['everlasting', 0.5022305458482222],
 'Fatty': ['honey', 0.3783582972103659],
 'Fielding': ['struck', 0.3801616763715929],
 'Giuseppe': ['mockingbird', 0.34876125002394576],
 'Gogh': ['mockingbird', 0.34432881114948055],
 'Goopyhead': ['Goopyhead', None],
 'Goya': ['mockingbird', 0.29851423169845465],
 'Italian': ['black', 0.38407057770847697],
 'Italy': ['Europe', 0.773761046700138],
 'Johnson': ['fox', 0.4

In [ ]:
tmp.doc2_mappings

{'10': ['one', 0.3317175871694564],
 '11': ['one', 0.21879580153031788],
 '12': ['one', 0.2567518056131179],
 '2': ['one', 0.285446516627334],
 '3': ['one', 0.29822437237068655],
 '4': ['one', 0.2696397992245068],
 '5': ['one', 0.2873463954950708],
 '6': ['one', 0.24819317337607122],
 '7': ['one', 0.2251424204803084],
 '8': ['one', 0.23220551910892778],
 '9': ['one', 0.21288253925923192],
 'Asia': ['Italy', 0.5791333897232351],
 'Europe': ['July', 0.5114580196931309],
 'God': ['angel', 0.5136990257769904],
 'Scatter': ['saunter', 0.48571356184156667],
 'Sufi': ['Stupids', 0.28166120194167854],
 'able': ['lucky', 0.5032188021681504],
 'adorable': ['little', 0.5430531677299464],
 'advice': ['tell', 0.488319196068744],
 'afternoon': ['tonight', 0.6662293013299185],
 'air': ['water', 0.5606072932226337],
 'ammonia': ['water', 0.4797829995928831],
 'anger': ['silence', 0.5172715619225122],
 'animal': ['fish', 0.5014694427222709],
 'ankle': ['neck', 0.5955336718182072],
 'announce': ['surpri

**CREATE NEW POEMS**

In [ ]:
#create new poems
tmp.Create_new_Poems()

In [ ]:
tmp.doc1_mappings

{'Baudelaire': ['mockingbird', 0.3210952234634215],
 'Bear': ['fox', 0.4937987093039136],
 'Beethoven': ['mockingbird', 0.333755009119292],
 'Beginners': ['great', 0.3132222177025964],
 'Big': ['sizzling', 0.5680608668918296],
 'Blinding': ['stare', 0.3897518063271421],
 'Bologna': ['scallop', 0.5245861323387164],
 'Cezanne': ['mockingbird', 0.29851423169845465],
 'Dopeheads': ['Dopeheads', None],
 'Dot': ['blue', 0.4339867192407223],
 'Dumb': ['silly', 0.7799713890983614],
 'Elba': ['mockingbird', 0.34876125002394576],
 'English': ['Europe', 0.45332710389385816],
 'Enlightenment': ['everlasting', 0.5022305458482222],
 'Fatty': ['honey', 0.3783582972103659],
 'Fielding': ['struck', 0.3801616763715929],
 'Giuseppe': ['mockingbird', 0.34876125002394576],
 'Gogh': ['mockingbird', 0.34432881114948055],
 'Goopyhead': ['Goopyhead', None],
 'Goya': ['mockingbird', 0.29851423169845465],
 'Italian': ['black', 0.38407057770847697],
 'Italy': ['Europe', 0.773761046700138],
 'Johnson': ['fox', 0.4

In [ ]:
for x in tmp.poet_1_details_df['new_poem']:
  print(str(x))


	 sister ' animal leave tree . 
	 tree long , tambourine tree 
	 tree luck sit home . 
	 turn luck . 

	 

	 sister ' animal leave tree . 
	 long door home 
	 mockingbird dance fire 
	 sit breath stone , 
	 cry , cry , tree , 

	 

	 sit jingle , 
	 mouth pay confidently mockingbird 
	 offer tree animal . 

	 

	 fall finally leave tree , 
	 sit dance page cry , 
	 nail luck dance 
	 sit jingle 

	 

	 stone sit diligent 
	 breath unforgettable cry animal rock , 
	 enormous poem trust 
	 mockingbird exciting imagination .
morning misery learn black fox mockingbird , leave luck black story black beautiful , thousand clothe want luck mountain , hardly want want Europe .

	 self - landscape mouth disturb : 
	 mockingbird dozen nail milk shadow , 
	 barn mockingbird sit moon float sky , 
	 tuffet leave soften work roof 
	 nail sit mockingbird . 

	 

	 mockingbird leave window 
	 leave mouth closet house 
	 call rug leave tall pebble . 

	 turn breath sit 
	 engage enormous helmet mouth 


In [ ]:
for x in tmp.poet_2_details_df['new_poem']:
  print(str(x))

second river . 
 green tonight green ! 
 torrid Poop , 
 miss door Bologna , 
 open , green fish , 
 sky hare , green green squirt neck — 
 turn , elbow , sit , 
 slide Bear green repertoire water sit . 
 like house 
 lucky thing , 
 one thing . 
 thing work , 
 holly , red , Bologna 
 green . 

 leave , slowly , understand scene .
1 . 

 great loud , pleasure scene . 

 sky , 
 red bowl , 
 green fish sky ; 

 sky autumn holder angel ; 

 green hat sky afterward , 

 tree , bird hemlock sky ; 

 Sampson , smokey , 
 Big Poop 
 branch wood tall spring , 
 sky green , 
 knock sky . 

 one . 

 imagine , , lucky house 
 rest - home home , home tour , 
 sit , , grownup breathe 
 thing home . 
 , . sky feed cloth door , 
 nest listen door , maybe 
 familiar spasm fish high , devastate , hoo - come 
 lucky sit little climb wild , 
 green smile . 
 , , breath fish , attention 
 fish ; - off neck water , 
 phrase water , appalling . 
 , , agreeable extraordinary , door high 
 arch , chair unp

**POET1 AND POET2 SIMILARITY for the newly generated poems**

In [ ]:
for x in tmp.poet_2_details_df['new_poem']:
  

**LDA**

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 28.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=634c943298bf9894f3eda0668c7e8ad7d38b7e6e6572136ad7478ae092347563
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
poet1=[]
poet2=[]
nlp=spacy.load('en_core_web_sm')

testing_df1 = tmp.poet_1_details_df
final1=[]
for x in testing_df1['cleaned']:
  test=[]
  for token in nlp(x):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        test.append(token.lemma_)
  final1.append(test)

testing_df2 = tmp.poet_2_details_df
final2=[]
for x in testing_df2['cleaned']:
  test=[]
  for token in nlp(x):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        test.append(token.lemma_)
  final2.append(test)

final3=[]
for x in testing_df1['new_poem']:
  test=[]
  for token in nlp(x):
    if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
      test.append(token.lemma_)
  final3.append(test)

final4=[]
for x in testing_df2['new_poem']:
  test=[]
  for token in nlp(x):
    if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
      test.append(token.lemma_)
  final4.append(test)

In [ ]:
#poet 1 all poems
id2word1 = corpora.Dictionary(final1)
corpus1 = []
for text in final1:
    new = id2word1.doc2bow(text)
    corpus1.append(new)

lda_model1 = gensim.models.LdaMulticore(corpus1, num_topics=10, id2word=id2word1, passes=2, workers=2)

In [ ]:
#poet 2 all poems

id2word2 = corpora.Dictionary(final2)
corpus2 = []
for text in final2:
    new = id2word2.doc2bow(text)
    corpus2.append(new)

lda_model2 = gensim.models.LdaMulticore(corpus2, num_topics=10, id2word=id2word2, passes=2, workers=2)

In [ ]:
#poet1-poet2 all poems

id2word3 = corpora.Dictionary(final3)
corpus3 = []
for text in final3:
    new = id2word3.doc2bow(text)
    corpus3.append(new)

lda_model3 = gensim.models.LdaMulticore(corpus3, num_topics=10, id2word=id2word3, passes=2, workers=2)

In [ ]:
#poet2-poet1 all poems

id2word4 = corpora.Dictionary(final4)
corpus4 = []
for text in final4:
    new = id2word4.doc2bow(text)
    corpus4.append(new)

lda_model4 = gensim.models.LdaMulticore(corpus4, num_topics=10, id2word=id2word4, passes=2, workers=2)

In [ ]:
#print topics and words poet1
for idx, topic in lda_model1.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.006*"hat" + 0.004*"head" + 0.004*"imagine" + 0.004*"candle" + 0.004*"Goya" + 0.004*"night" + 0.004*"work" + 0.003*"see" + 0.003*"room" + 0.003*"door"
Topic: 1 
Words: 0.008*"little" + 0.008*"eat" + 0.008*"car" + 0.008*"tour" + 0.008*"head" + 0.008*"way" + 0.007*"bed" + 0.007*"know" + 0.006*"giant" + 0.006*"fish"
Topic: 2 
Words: 0.068*"bark" + 0.030*"dog" + 0.023*"Beethoven" + 0.023*"barking" + 0.016*"sit" + 0.016*"neighbor" + 0.016*"house" + 0.015*"stop" + 0.009*"way" + 0.009*"head"
Topic: 3 
Words: 0.013*"time" + 0.013*"afternoon" + 0.013*"speak" + 0.013*"train" + 0.013*"one" + 0.013*"communicate" + 0.013*"shop" + 0.013*"Beginners" + 0.013*"male" + 0.013*"resemble"
Topic: 4 
Words: 0.030*"Susquehanna" + 0.029*"fish" + 0.013*"little" + 0.013*"river" + 0.012*"paint" + 0.012*"water" + 0.012*"pleasure" + 0.012*"boat" + 0.008*"come" + 0.008*"imagine"
Topic: 5 
Words: 0.013*"close" + 0.013*"dark" + 0.013*"book" + 0.013*"consider" + 0.012*"call" + 0.012*"stand" + 0.012*"p

In [ ]:
#print topics and words poet1
final=set()
for index, topic in lda_model1.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet 1 important Topic words\n')
print(final)

Poet 1 important Topic words

{'fish', 'Susquehanna', 'tailor', 'neck', 'giant', 'camera', 'record', 'head', 'resemble', 'sure', 'ruffle', 'release', 'breathe', 'ghost', 'work', 'door', 'sit', 'fix', 'struggle', 'concert', 'parlor', 'memorize', 'thing', 'tall', 'body', 'traveler', 'wit', 'grasp', 'green', 'arm', 'Beginners', 'eye', 'tree', 'carry', 'fielding', 'Bear', 'country', 'cold', 'attentive', 'soft', 'grin', 'Rembrant', 'box', 'doorway', 'pleasure', 'insistent', 'bed', 'slide', 'shop', 'agreeable', 'English', 'listen', 'water', 'bird', 'summon', 'pen', 'match', 'cake', 'fingernail', 'imagine', 'respectful', 'turn', 'rapid', 'morne', 'nee', 'element', 'little', 'shadow', 'fellow', 'hat', 'tease', 'camper', 'smokey', 'orchestra', 'stand', 'rioting', 'ranger', 'Goya', 'italian', 'innovative', 'red', 'enigma', 'car', 'famous', 'spacious', 'inquire', 'hand', 'call', 'disturb', 'dining', 'musician', 'fit', 'Roberto', 'canvas', 'flap', 'tour', 'embrace', 'dog', 'vanish', 'hall', 'dark'

In [ ]:
#print topics and words poet2

for idx, topic in lda_model2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.019*"word" + 0.011*"begin" + 0.011*"open" + 0.011*"morning" + 0.010*"little" + 0.010*"blue" + 0.010*"pale" + 0.010*"edge" + 0.010*"schoolhouse" + 0.010*"shine"
Topic: 1 
Words: 0.020*"air" + 0.014*"earth" + 0.014*"world" + 0.014*"body" + 0.014*"love" + 0.014*"go" + 0.014*"time" + 0.014*"wild" + 0.013*"goose" + 0.013*"tell"
Topic: 2 
Words: 0.018*"wind" + 0.012*"tree" + 0.011*"leave" + 0.010*"sleep" + 0.010*"field" + 0.010*"earth" + 0.010*"long" + 0.010*"little" + 0.010*"begin" + 0.010*"look"
Topic: 3 
Words: 0.004*"world" + 0.004*"flower" + 0.004*"poem" + 0.003*"earth" + 0.003*"tree" + 0.003*"wind" + 0.003*"house" + 0.003*"barn" + 0.003*"good" + 0.003*"child"
Topic: 4 
Words: 0.022*"voice" + 0.017*"little" + 0.017*"world" + 0.017*"begin" + 0.016*"wind" + 0.012*"know" + 0.012*"look" + 0.012*"tree" + 0.011*"life" + 0.011*"keep"
Topic: 5 
Words: 0.013*"poem" + 0.012*"world" + 0.010*"child" + 0.010*"green" + 0.010*"earth" + 0.010*"house" + 0.010*"life" + 0.010*"flower" +

In [ ]:
#print topics and words poet2
final=set()
for index, topic in lda_model2.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet 2 important Topic words\n')
print(final)

Poet 2 important Topic words

{'mossy', 'work', 'leave', 'whisper', 'tattere', 'mouth', 'white', 'thing', 'wind', 'body', 'lifetime', 'long', 'thumb', 'green', 'animal', 'secret', 'arm', 'tree', 'eye', 'taper', 'carry', 'enormous', 'responsibility', 'endless', 'mountain', 'morning', 'save', 'hay', 'bury', 'fall', 'listen', 'mortal', 'goose', 'open', 'water', 'God', 'bird', 'gray', 'gull', 'turn', 'run', 'honey', 'beetle', 'little', 'voice', 'schoolhouse', 'cupboard', 'lift', 'shake', 'mean', 'ring', 'pond', 'fragrance', 'scallop', 'shoulder', 'tuffet', 'learn', 'praise', 'evening', 'rich', 'mussel', 'everlasting', 'bring', 'dark', 'split', 'think', 'fulfillment', 'walk', 'cattail', 'river', 'air', 'talk', 'sea', 'come', 'earth', 'story', 'mile', 'bear', 'pillar', 'finger', 'meaning', 'wild', 'field', 'love', 'day', 'family', 'dozen', 'good', 'thousand', 'flower', 'emperor', 'word', 'shut', 'world', 'lack', 'egg', 'deep', 'moisture', 'live', 'house', 'useful', 'place', 'sweetness', 'tab

In [ ]:
#print topics and words poet1-poet2

for idx, topic in lda_model3.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.048*"sit" + 0.043*"white" + 0.039*"luck" + 0.037*"leave" + 0.028*"long" + 0.024*"Europe" + 0.023*"morning" + 0.022*"sky" + 0.020*"hummingbird" + 0.020*"work"
Topic: 1 
Words: 0.009*"luck" + 0.008*"leave" + 0.007*"work" + 0.007*"mouth" + 0.007*"sit" + 0.006*"mockingbird" + 0.006*"helmet" + 0.006*"nail" + 0.006*"morning" + 0.006*"home"
Topic: 2 
Words: 0.072*"want" + 0.072*"black" + 0.049*"luck" + 0.026*"clothe" + 0.026*"fox" + 0.026*"beautiful" + 0.026*"learn" + 0.026*"misery" + 0.026*"mockingbird" + 0.026*"mountain"
Topic: 3 
Words: 0.071*"tree" + 0.071*"sit" + 0.035*"luck" + 0.035*"leave" + 0.033*"cry" + 0.033*"animal" + 0.028*"mockingbird" + 0.024*"dance" + 0.021*"door" + 0.019*"breath"
Topic: 4 
Words: 0.005*"sit" + 0.005*"thing" + 0.005*"silly" + 0.005*"work" + 0.005*"little" + 0.005*"luck" + 0.004*"leave" + 0.004*"long" + 0.004*"want" + 0.004*"fox"
Topic: 5 
Words: 0.047*"thing" + 0.040*"sit" + 0.038*"leave" + 0.036*"door" + 0.024*"clothe" + 0.023*"long" + 0.022

In [ ]:
#print topics and words
final=set()
for index, topic in lda_model3.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet1-poet2 important Topic words\n')
print(final)

Poet1-poet2 important Topic words

{'feather', 'luck', 'home', 'hinge', 'self', 'ankle', 'outward', 'hummingbird', 'milk', 'forearm', 'beautiful', 'breath', 'knee', 'work', 'door', 'sit', 'desert', 'leave', 'sky', 'jingle', 'white', 'mouth', 'want', 'misery', 'thing', 'sunrise', 'honeysuckle', 'touch', 'lover', 'tall', 'nail', 'page', 'long', 'animal', 'climb', 'tree', 'visit', 'carry', 'enormous', 'sister', 'tambourine', 'glare', 'kneel', 'soft', 'mountain', 'morning', 'longing', 'temple', 'fall', 'exciting', 'water', 'trust', 'jug', 'turn', 'little', 'raft', 'Sewerface', 'landscape', 'step', 'thank', 'fox', 'cry', 'offer', 'theater', 'shake', 'soften', 'lantern', 'pay', 'ring', 'explanation', 'silly', 'fly', 'disturb', 'shoulder', 'tuffet', 'learn', 'evening', 'stone', 'talk', 'story', 'frisky', 'muscular', 'finger', 'pebble', 'Europe', 'thousand', 'emperor', 'dance', 'kitchen', 'strike', 'arch', 'moisture', 'house', 'curve', 'black', 'moon', 'clothe', 'mockingbird', 'cusp', 'curved'

In [ ]:
#print topics and words Poet2-poet1

for idx, topic in lda_model4.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.077*"tree" + 0.059*"fish" + 0.038*"leave" + 0.032*"green" + 0.032*"neck" + 0.028*"tonight" + 0.022*"smile" + 0.022*"sit" + 0.020*"extraordinary" + 0.020*"climb"
Topic: 1 
Words: 0.027*"tree" + 0.024*"green" + 0.021*"sky" + 0.019*"sit" + 0.019*"fish" + 0.019*"thing" + 0.018*"neck" + 0.016*"smile" + 0.013*"lucky" + 0.012*"home"
Topic: 2 
Words: 0.079*"sit" + 0.076*"work" + 0.045*"neck" + 0.043*"fish" + 0.043*"thing" + 0.037*"green" + 0.036*"smile" + 0.031*"sky" + 0.027*"tree" + 0.020*"leave"
Topic: 3 
Words: 0.011*"green" + 0.011*"sit" + 0.010*"sky" + 0.009*"thing" + 0.009*"one" + 0.009*"work" + 0.008*"fish" + 0.008*"lucky" + 0.008*"home" + 0.007*"door"
Topic: 4 
Words: 0.031*"fish" + 0.029*"tree" + 0.020*"neck" + 0.020*"smile" + 0.019*"green" + 0.017*"sit" + 0.016*"home" + 0.014*"leave" + 0.014*"tonight" + 0.013*"work"
Topic: 5 
Words: 0.074*"work" + 0.046*"sit" + 0.040*"tree" + 0.032*"green" + 0.032*"fish" + 0.030*"smile" + 0.029*"neck" + 0.027*"end" + 0.020*"thing" 

In [ ]:
#print topics and words
final=set()
for index, topic in lda_model3.show_topics(formatted=False, num_words= 30):
  for x in topic:
    final.add(x[0])

print('Poet2-poet1 important Topic words\n')
print(final)

Poet2-poet1 important Topic words

{'feather', 'luck', 'home', 'hinge', 'self', 'ankle', 'outward', 'hummingbird', 'milk', 'forearm', 'beautiful', 'breath', 'knee', 'work', 'door', 'sit', 'desert', 'leave', 'sky', 'jingle', 'white', 'mouth', 'want', 'misery', 'thing', 'sunrise', 'honeysuckle', 'touch', 'lover', 'tall', 'nail', 'page', 'long', 'animal', 'climb', 'tree', 'visit', 'carry', 'enormous', 'sister', 'tambourine', 'glare', 'kneel', 'soft', 'mountain', 'morning', 'longing', 'temple', 'fall', 'exciting', 'water', 'trust', 'jug', 'turn', 'little', 'raft', 'Sewerface', 'landscape', 'step', 'thank', 'fox', 'cry', 'offer', 'theater', 'shake', 'soften', 'lantern', 'pay', 'ring', 'explanation', 'silly', 'fly', 'disturb', 'shoulder', 'tuffet', 'learn', 'evening', 'stone', 'talk', 'story', 'frisky', 'muscular', 'finger', 'pebble', 'Europe', 'thousand', 'emperor', 'dance', 'kitchen', 'strike', 'arch', 'moisture', 'house', 'curve', 'black', 'moon', 'clothe', 'mockingbird', 'cusp', 'curved'

In [ ]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model1, corpus1, id2word1)
vis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.051704  0.095449       1        1  31.668362
6      0.137684 -0.038287       2        1  16.545782
9     -0.113588 -0.079173       3        1  15.119367
4      0.081126  0.087452       4        1  13.122415
2     -0.067036  0.049282       5        1   9.128784
8      0.042134 -0.073950       6        1   8.707764
3     -0.006416 -0.008823       7        1   2.856633
5     -0.015103 -0.015393       8        1   2.572041
7     -0.016491 -0.015638       9        1   0.139426
0      0.009395 -0.000919      10        1   0.139426, topic_info=            Term      Freq     Total Category  logprob  loglift
1           bark  4.000000  4.000000  Default  30.0000  30.0000
463  Susquehanna  3.000000  3.000000  Default  29.0000  29.0000
199         fish  4.000000  4.000000  Default  28.0000  28.0000
108          hat  3.000000  3.000000  Default  27.0000  27.0000
8            dog  2.000000  2.000000  Default  26.0000  26.0000
..           ...       ...       ...      ...      ...      ...
14        famous  0.002551  2.499137  Topic10  -5.9485  -0.3119
154         wife  0.002456  1.952522  Topic10  -5.9865  -0.1031
13           eye  0.002505  3.072742  Topic10  -5.9664  -0.5364
121     memorize  0.002409  2.036887  Topic10  -6.0057  -0.1646
203        giant  0.002400  2.044527  Topic10  -6.0092  -0.1719

[497 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         5  0.524352  Beethoven
64        2  0.890761   Blinding
333       1  0.733064    Bologna
334       1  0.735591        Dot
66        2  0.889509       Gogh
...     ...       ...        ...
429       1  0.640693       word
156       1  0.379678       work
156       2  0.379678       work
460       6  1.070305       wrap
332       3  0.988220       yarn

[358 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 10, 5, 3, 9, 4, 6, 8, 1])

In [ ]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, corpus2, id2word2)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.112335  0.074479       1        1  41.583590
4      0.158576  0.009206       2        1  15.123673
8     -0.038291 -0.149271       3        1  14.880010
1      0.015592  0.045205       4        1  10.978771
2     -0.022901 -0.019887       5        1   6.097821
9      0.006566 -0.015504       6        1   5.854017
0      0.016315  0.025691       7        1   5.122656
7     -0.003340  0.012381       8        1   0.119820
6     -0.018536  0.012793       9        1   0.119820
3     -0.001646  0.004907      10        1   0.119820, topic_info=       Term      Freq     Total Category  logprob  loglift
198    poem  5.000000  5.000000  Default  30.0000  30.0000
289   world  9.000000  9.000000  Default  29.0000  29.0000
136   green  4.000000  4.000000  Default  28.0000  28.0000
86    child  4.000000  4.000000  Default  27.0000  27.0000
111   earth  5.000000  5.000000  Default  26.0000  26.0000
..      ...       ...       ...      ...      ...      ...
15   little  0.002632  5.760557  Topic10  -5.9224  -0.9642
163    life  0.002614  6.567387  Topic10  -5.9292  -1.1020
168    long  0.002527  4.104268  Topic10  -5.9629  -0.6656
156    know  0.002539  7.340626  Topic10  -5.9582  -1.2423
314    look  0.002478  3.891476  Topic10  -5.9826  -0.6321

[591 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
46        1  0.739513       3
458       2  0.906008    Asia
459       2  0.902737  Europe
53        1  0.927772     God
460       2  0.904940    Sufi
...     ...       ...     ...
289       1  0.430423   world
289       2  0.215212   world
289       3  0.107606   world
289       4  0.107606   world
495       2  0.905270    yard

[337 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 9, 2, 3, 10, 1, 8, 7, 4])

In [ ]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model3, corpus3, id2word3)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.055995 -0.066225       1        1  48.454963
5      0.061240  0.101780       2        1  23.315180
7      0.063542 -0.061490       3        1  15.576696
3      0.083857  0.009053       4        1   9.068624
2     -0.088066 -0.020116       5        1   2.881314
6     -0.013206  0.021413       6        1   0.140645
9     -0.025033 -0.002804       7        1   0.140645
1     -0.034999  0.000909       8        1   0.140644
4     -0.050837  0.008242       9        1   0.140644
8     -0.052494  0.009238      10        1   0.140644, topic_info=        Term      Freq      Total Category  logprob  loglift
25       sit  32.00000  32.000000  Default  30.0000  30.0000
15      luck  22.00000  22.000000  Default  29.0000  29.0000
82     white  17.00000  17.000000  Default  28.0000  28.0000
32    Europe  11.00000  11.000000  Default  27.0000  27.0000
39   morning  12.00000  12.000000  Default  26.0000  26.0000
..       ...       ...        ...      ...      ...      ...
2        cry   0.00425   3.202659  Topic10  -5.4381  -0.0581
37     learn   0.00425   2.644836  Topic10  -5.4381   0.1332
38    misery   0.00425   1.667854  Topic10  -5.4381   0.5943
1     breath   0.00425   6.847744  Topic10  -5.4381  -0.8181
115    silly   0.00425   7.382022  Topic10  -5.4381  -0.8932

[573 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
85        3  0.837644  Dopeheads
32        1  0.695238     Europe
32        2  0.173809     Europe
32        5  0.086905     Europe
87        3  0.837174  Goopyhead
...     ...       ...        ...
83        1  0.757578     window
193       2  0.781910       word
84        1  0.579718       work
84        2  0.082817       work
84        3  0.331267       work

[309 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 8, 4, 3, 7, 10, 2, 5, 9])

In [ ]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model4, corpus4, id2word4)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.019806  0.047935       1        1  47.978657
9      0.077508  0.001643       2        1  20.414817
2      0.110029 -0.001122       3        1  10.827387
0     -0.028778 -0.126150       4        1   9.347537
5      0.035861 -0.023221       5        1   5.904411
6     -0.009574  0.083495       6        1   5.043821
4     -0.027584 -0.030462       7        1   0.120845
1     -0.028380 -0.004155       8        1   0.120845
8     -0.084485  0.027805       9        1   0.120840
3     -0.064402  0.024232      10        1   0.120840, topic_info=        Term       Freq      Total Category  logprob  loglift
6      green  32.000000  32.000000  Default  30.0000  30.0000
136     tree  24.000000  24.000000  Default  29.0000  29.0000
20       sit  43.000000  43.000000  Default  28.0000  28.0000
5       fish  24.000000  24.000000  Default  27.0000  27.0000
30      work  32.000000  32.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
53   breathe   0.005204   5.881243  Topic10  -5.2406  -0.3117
45       big   0.005538  12.326460  Topic10  -5.1784  -0.9894
121  silence   0.005241   7.155825  Topic10  -5.2336  -0.5008
123    smile   0.005625  22.180897  Topic10  -5.1629  -1.5614
125     soft   0.005195   7.761055  Topic10  -5.2423  -0.5907

[564 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         2  0.422705           Bear
0         5  0.422705           Bear
31        1  0.660328            Big
1         6  0.730173        Bologna
175       2  0.804936  Enlightenment
...     ...       ...            ...
30        3  0.216626           work
30        5  0.123786           work
30        6  0.030947           work
202       5  0.878605         work--
157       4  0.833265           wrap

[344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 3, 1, 6, 7, 5, 2, 9, 4])

**GSDMM**

In [ ]:
!pip install git+https://github.com/rwalk/gsdmm.git

  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-nelsa8li
  Running command git clone -q https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-nelsa8li
  Created wheel for gsdmm: filename=gsdmm-0.1-py3-none-any.whl size=4602 sha256=4cad01ca4863090dceef4fde6d0bc151baf2a00a93e1ae40d64270ef57cdb971
  Stored in directory: /tmp/pip-ephem-wheel-cache-wm6to3vh/wheels/34/65/a6/7eef67b88abae954fecd22587bd755c27b58a9ffe488d6b0de
Successfully built gsdmm


In [ ]:
from gsdmm import MovieGroupProcess

In [ ]:
mgp = MovieGroupProcess(K=3, alpha=0.01, beta=0.01, n_iters=30)
# create variable containing length of dictionary/vocab
vocab = set(x for y in final1 for x in y)
n_terms = len(vocab)
n_docs = 10

In [ ]:
model = mgp.fit(final1, n_terms)

In stage 0: transferred 6 clusters with 3 clusters populated
In stage 1: transferred 2 clusters with 3 clusters populated
In stage 2: transferred 2 clusters with 3 clusters populated
In stage 3: transferred 0 clusters with 3 clusters populated
In stage 4: transferred 0 clusters with 3 clusters populated
In stage 5: transferred 0 clusters with 3 clusters populated
In stage 6: transferred 0 clusters with 3 clusters populated
In stage 7: transferred 0 clusters with 3 clusters populated
In stage 8: transferred 0 clusters with 3 clusters populated
In stage 9: transferred 0 clusters with 3 clusters populated
In stage 10: transferred 0 clusters with 3 clusters populated
In stage 11: transferred 0 clusters with 3 clusters populated
In stage 12: transferred 0 clusters with 3 clusters populated
In stage 13: transferred 0 clusters with 3 clusters populated
In stage 14: transferred 0 clusters with 3 clusters populated
In stage 15: transferred 0 clusters with 3 clusters populated
In stage 16: trans

In [ ]:
# print number of documents per topic
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-15:][::-1]
print('Most important clusters (by number of docs inside):', top_index)

# define function to get top words per topic
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster, sort_dicts))

# get top words in topics
top_words(mgp.cluster_word_distribution, top_index, 20)

Number of documents per topic : [3 2 5]
Most important clusters (by number of docs inside): [2 0 1]

Cluster 2 : [('fish', 6), ('head', 5), ('imagine', 5), ('Susquehanna', 5), ('paint', 4), ('night', 4), ('room', 4), ('light', 4), ('door', 4), ('little', 4), ('hat', 4), ('eye', 3), ('stand', 3), ('know', 3), ('work', 3), ('come', 3), ('try', 3), ('small', 3), ('river', 3), ('Goya', 3)]

Cluster 0 : [('look', 2), ('afternoon', 1), ('struggle', 1), ('communicate', 1), ('italian', 1), ('Roberto', 1), ('Giuseppe', 1), ('begin', 1), ('resemble', 1), ('male', 1), ('character', 1), ('Italian', 1), ('Beginners', 1), ('one', 1), ('shop', 1), ('inquire', 1), ('time', 1), ('train', 1), ('speak', 1), ('write', 1)]

Cluster 1 : [('bark', 9), ('dog', 4), ('Beethoven', 3), ('barking', 3), ('high', 2), ('head', 2), ('neighbor', 2), ('stop', 2), ('house', 2), ('sit', 2), ('rhythmic', 1), ('time', 1), ('leave', 1), ('switch', 1), ('way', 1), ('close', 1), ('window', 1), ('symphony', 1), ('blast', 1), ('

**TEXT SUMMARIZATION**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
poem1_poem2_summary=[]
with tf.device('/device:GPU:0'):
  for x in tmp.poet_1_details_df['new_poem']:
    ARTICLE_TO_SUMMARIZE = str(x)
    inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=512, return_tensors="pt")
    summary_ids = model.generate(inputs["input_ids"])
    poem1_poem2_summary.append(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

In [ ]:
poem1_poem2_summary

['mockingbird dance fire sit breath stone , cry , cry , tree , sit jingle , mouth pay confidently mockingbird offer tree animal . long door home mockingbird dance fire sit breath stone , cry , cry , tree , sit jingle , mouth pay confidently mockingbird',
 'Morning misery learn black fox mockingbird , leave luck mountain , thousand clothe want luck mountain , hardly want want Europe .',
 '" work , " " nail , " leave door leave finger nail , luck burn unforgettable lantern helmet .',
 'silly silly luck : long hand , road ring , gloom mountain hurry , sit sister breath , disturb want fox kitchen terrible work , man long honey God , lover little Dopeheads Mr.',
 'hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingbird hummingb

In [ ]:
poem2_poem1_summary=[]

with tf.device('/device:GPU:0'):
  for x in tmp.poet_2_details_df['new_poem']:
    ARTICLE_TO_SUMMARIZE = str(x)
    inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=512,return_tensors="pt")
    summary_ids = model.generate(inputs["input_ids"])
    poem2_poem1_summary.append(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

In [ ]:
poem2_poem1_summary

['green tonight green ! torrid Poop , miss door Bologna , open , green fish , sky hare , green green squirt neck — turn , elbow , sit , slide Bear green repertoire water sit .',
 'sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit , sit ',
 "Tonight tell - dog ; smile door crumble - fish , shop tree's soft basilica .",
 'Trees sit neck pillar green , sit dense cake establish , big angle forest boat away green neck fish , work , month.',
 'Work , work , work , work , work , work , work , work , work , work , work , work , work , work , work , work , work , work , work , work , work ',
 'sit sit sit work soft neck sky instead endless water endless autumn sky ?',
 '" lucky lucky , sit baritone , scene leave lose paper sky , Big baritone slowly grasp , sit owner alertness dark dark thing , work save home save .',
 'Work , work , work , work , work , work , work , work , work , work , work , work , work , work , work , work 

**WRITE_TO_JSON**

In [ ]:
poet1_details=tmp.poet1_details
poet1_details['new_poem'] = tmp.new_poem1_poem2
poet1_details['POS']=tmp.doc1_pos
poet1_details['summary']=poem1_poem2_summary

poet2_details=tmp.poet2_details
poet2_details['new_poem'] = tmp.new_poem1_poem2
poet2_details['POS']=tmp.doc2_pos
poet2_details['summary'] = poem2_poem1_summary

In [ ]:
def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

import json
with open("poet1.json", "w") as outfile:
  poet1 = json.dumps(poet1_details,default=set_default)
  outfile.write(poet1)

with open("poet2.json", "w") as outfile:
  poet2 = json.dumps(poet2_details,default=set_default)
  outfile.write(poet2)

**GET POET DETAILS FROM POET WIKI PAGE INSTEAD NEWS ARTICLES**



In [ ]:
! pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=776124e56a39db6c3e37a0c3c179b45de199d7edd2d99ff838380cb799b43c65
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [ ]:
import wikipedia

In [ ]:
def Retrive(top_n=1):
  '''
  This Function returns Wikipedia Pages results based on user query
  '''
  search_results_pages=[]
  for x in range(2):
    search_text = input('Enter Poet Name')
    search_text = search_text.strip()
    search_results = wikipedia.search(search_text, results=top_n)
    for docs in search_results:
        search_results_pages.append(wikipedia.page(docs,
                auto_suggest=False))
  return search_results_pages

In [ ]:
pages = Retrive()

Enter Poet Namebilly collins
Enter Poet Namemary j oliver


In [ ]:
poet_1 = pages[0].content

In [ ]:
poet_2 = pages[1].content

In [ ]:
def lemma_pos(corpus):
  preprocesed  = re.sub(r"[^a-zA-Z0-9]+", ' ', corpus)
  nlp=spacy.load('en_core_web_sm')
  lemmatized = []
  for token in nlp(preprocesed):
      if token.pos_ in ('PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'):
        lemmatized.append(token.lemma_)
  return lemmatized

In [ ]:
poet_1_lemma_corpus = lemma_pos(poet_1)
poet_2_lemma_corpus = lemma_pos(poet_2)

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

In [ ]:
print(poet_1_lemma_corpus, end='')

['William', 'James', 'Collins', 'bear', 'March', '22', '1941', 'american', 'poet', 'appoint', 'Poet', 'Laureate', 'United', 'States', '2001', '2003', 'Distinguished', 'Professor', 'Lehman', 'College', 'City', 'University', 'New', 'York', 'retire', '2016', 'Collins', 'recognize', 'Literary', 'Lion', 'New', 'York', 'Public', 'Library', '1992', 'select', 'New', 'York', 'State', 'Poet', '2004', '2006', '2016', 'Collins', 'induct', 'American', 'Academy', 'Arts', 'Letters', '2020', 'teacher', 'MFA', 'program', 'Stony', 'Brook', 'Southampton', 'early', 'life', 'education', 'Collins', 'bear', 'Manhattan', 'William', 'Katherine', 'Collins', 'grow', 'Queens', 'White', 'Plains', 'William', 'bear', 'large', 'family', 'Ireland', 'Katherine', 'Canada', 'mother', 'Katherine', 'Collins', 'nurse', 'stop', 'work', 'raise', 'couple', 's', 'only', 'child', 'Mrs', 'Collins', 'ability', 'recite', 'verse', 'subject', 'cultivate', 'young', 'son', 'love', 'word', 'write', 'speak', 'Collins', 'attend', 'Archbis

In [ ]:
print(poet_2_lemma_corpus, end='')

['Mary', 'Jane', 'Oliver', 'September', '10', '1935', 'January', '17', '2019', 'american', 'poet', 'win', 'National', 'Book', 'Award', 'Pulitzer', 'Prize', 'work', 'inspire', 'nature', 'human', 'world', 'stem', 'lifelong', 'passion', 'solitary', 'walk', 'wild', 'characterise', 'sincere', 'wonderment', 'impact', 'natural', 'imagery', 'convey', 'unadorned', 'language', '2007', 'declare', 'country', 'good', 'sell', 'poet', 'early', 'life', 'Mary', 'Oliver', 'bear', 'Edward', 'William', 'Helen', 'M', 'Vlasak', 'Oliver', 'September', '10', '1935', 'Maple', 'Heights', 'Ohio', 'semi', 'rural', 'suburb', 'Cleveland', 'father', 'social', 'study', 'teacher', 'athletic', 'coach', 'Cleveland', 'public', 'school', 'child', 'spend', 'great', 'deal', 'time', 'enjoy', 'go', 'walk', 'read', 'interview', 'Christian', 'Science', 'Monitor', '1992', 'Oliver', 'comment', 'grow', 'Ohio', 'say', 'pastoral', 'nice', 'extended', 'family', 'don', 't', 'know', 'feel', 'affinity', 'natural', 'world', 'available', 

**POET1**

In [ ]:
final_corpus1=[]
id2wordd1 = corpora.Dictionary([poet_1_lemma_corpus])
for text in [poet_1_lemma_corpus]:
    new = id2wordd1.doc2bow(text)
    final_corpus1.append(new)

In [ ]:
lda_modelpoet1 = gensim.models.LdaMulticore(final_corpus1, num_topics=2, id2word=id2wordd1, passes=2, workers=2)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


In [ ]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_modelpoet1, final_corpus1, id2wordd1)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.006553  0.0       1        1  99.944109
0     -0.006553  0.0       2        1   0.055891, topic_info=        Term       Freq      Total Category  logprob  loglift
108  Collins  43.000000  43.000000  Default  30.0000  30.0000
500     poem  20.000000  20.000000  Default  29.0000  29.0000
321     York  24.000000  24.000000  Default  28.0000  28.0000
219      New  25.000000  25.000000  Default  27.0000  27.0000
243   Poetry  13.000000  13.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
401     edit   0.001751   4.276514   Topic2  -5.9414  -0.3111
444  include   0.001801   5.148356   Topic2  -5.9131  -0.4684
68      Arts   0.001788   5.156891   Topic2  -5.9203  -0.4773
579     time   0.001777   6.069341   Topic2  -5.9268  -0.6466
298      TED   0.001754   5.179303   Topic2  -5.9396  -0.5008

[135 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
3         1  1.002279     180
23        1  1.028802    2001
27        1  1.036802    2005
28        1  0.894056    2006
34        1  1.030564    2013
...     ...       ...     ...
548       1  0.844496  settle
569       1  0.840044    such
579       1  0.988575    time
584       1  0.844422    turn
601       1  1.192570    work

[78 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [ ]:
for index, topic in lda_modelpoet1.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

Topic: 9 
Words: ['Collins', 'poem', 'York', 'New', 'Random', 'Poetry', 'House', 'Poet', 'poetry', 'Billy', 'Prize', 'Laureate', 'Press', 'University', '2005', 'poet', 'S', 'College', 'American', '2013', '180', '2001', 'collin', 'Names', '2016', 'include', 'Arts', 'time', 'TED', 'edit']
Topic: 9 
Words: ['Collins', 'New', 'York', 'poem', 'Poetry', 'Billy', 'House', 'poetry', 'Random', 'Poet', '2001', '2013', 'University', 'Press', 'Laureate', 'Prize', 'time', 'poet', '2005', 'U', 'TED', 'Arts', 'include', 'collin', '180', 'American', 'College', 'S', '2006', 'read']


In [ ]:
#print topics and words
poet1_history=set()
for index, topic in lda_modelpoet1.show_topics(formatted=False, num_words= 30):
  for x in topic:
    poet1_history.add(x[0])

print('Poet1 important Topic from History/Wiki \n')
print(poet1_history)

Poet1 important Topic from History/Wiki 

{'Billy', 'New', 'House', 'Random', 'poetry', 'edit', 'poet', 'Laureate', 'TED', 'American', 'Press', 'Collins', '2006', '2001', 'U', 'York', 'University', 'Poetry', 'Poet', 'College', '2013', 'Names', 'poem', 'Prize', 'S', 'Arts', '2005', '180', 'time', 'collin', 'include', '2016', 'read'}


**POET2**

In [ ]:
final_corpus2=[]
id2wordd2 = corpora.Dictionary([poet_2_lemma_corpus])
for text in [poet_2_lemma_corpus]:
    new = id2wordd2.doc2bow(text)
    final_corpus2.append(new)

In [ ]:
lda_modelpoet2 = gensim.models.LdaMulticore(final_corpus2, num_topics=2, id2word=id2wordd2, passes=2, workers=2)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


In [ ]:
#Visualize topics
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_modelpoet2, final_corpus2, id2wordd2)
vis

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.009936  0.0       1        1  99.944905
1     -0.009936  0.0       2        1   0.055095, topic_info=           Term       Freq      Total Category  logprob  loglift
191      Oliver  33.000000  33.000000  Default  30.0000  30.0000
170        Mary  11.000000  11.000000  Default  29.0000  29.0000
57     American   8.000000   8.000000  Default  28.0000  28.0000
657       world   9.000000   9.000000  Default  27.0000  27.0000
520        poet   7.000000   7.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
396       first   0.001253   4.200405   Topic2  -6.1852  -0.6137
98         Cook   0.001274   5.004767   Topic2  -6.1685  -0.7722
331  collection   0.001238   4.208700   Topic2  -6.1972  -0.6277
659       write   0.001230   4.213189   Topic2  -6.2038  -0.6353
457        life   0.001246   5.020502   Topic2  -6.1910  -0.7978

[131 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3         1  0.876702       15
4         1  1.142057       17
7         1  0.877425     1951
18        1  1.021138     1992
53        1  0.875219  Academy
...     ...       ...      ...
650       1  1.012658      win
652       1  0.881280    woman
656       1  0.948478     work
657       1  1.038781    world
659       1  0.949400    write

[75 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [ ]:
for index, topic in lda_modelpoet2.show_topics(formatted=False, num_words= 30):
    print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))

Topic: 9 
Words: ['Oliver', 'Mary', 'world', 'Poetry', 'American', 'New', 'poet', 'Award', 'nature', 'poem', 'work', 'Poems', 'poetry', 'life', 'Cook', 'describe', 'say', 'know', 'win', 'natural', 'walk', 'University', '1992', 'interview', 'Pulitzer', 'National', 'make', 'Honorary', 'write', 'collection']
Topic: 9 
Words: ['Oliver', 'Mary', 'American', 'work', 'poet', 'world', 'nature', 'poem', 'New', 'Award', 'Poetry', '1992', 'University', 'walk', 'natural', 'poetry', 'win', 'know', 'say', 'Poems', 'Prize', 'describe', 'Review', 'Cook', 'Ohio', 'first', 'life', 'collection', 'write', 'Doctorate']


In [ ]:
#print topics and words
poet2_history=set()
for index, topic in lda_modelpoet2.show_topics(formatted=False, num_words= 30):
  for x in topic:
    poet2_history.add(x[0])

print('Poet2 important Topic from History/Wiki \n')
print(poet2_history)

Poet2 important Topic from History/Wiki 

{'Review', 'New', 'poetry', '1992', 'poet', 'work', 'interview', 'National', 'American', 'nature', 'know', 'life', 'win', 'Cook', 'Honorary', 'Award', 'University', 'Poetry', 'make', 'Oliver', 'world', 'say', 'natural', 'Mary', 'Ohio', 'Pulitzer', 'poem', 'Prize', 'write', 'walk', 'describe', 'first', 'Doctorate', 'collection', 'Poems'}
